# Module 4: AprilTags

Our camera is now able to detect images and perform basic modifications to the frame captured. With this, we will now use a library to detect specific markers known as "AprilTags". They are similar to QR codes, in that they contain information using black and white squares on a 2D image (such as a paper), but are much smaller, to improve detectability and robustness.

This module should follow Module 3: Camera and Computer Vision Basics

Here are the normal packages we need to import in our code

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import time

robot = Robot()
camera = Camera.instance()

Now we have a new one, known as duckietown april tags! We create a "detector" that will be used to analyze the image taken from the camera to find any april tags.

In [ ]:
import dt_apriltags

detector = dt_apriltags.Detector()

This following loop will detect AprilTags located in the frame of the camera, draw a box around the tag detected, and display the apriltag (and its ID). The individual lines will be explored in the block after the code.

In [ ]:
while True:
    clear_output(wait = True)
    image = np.array(camera.value)
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    detections = detector.detect(gray_frame)

    for detection in detections:
        corners = detection.corners
        corners = [(int(x), int(y)) for (x, y) in corners]

        cv2.polylines(image, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)

        tag_id = detection.tag_id
        cv2.putText(image, f"ID: {tag_id}", (int(corners[0][0]), int(corners[0][1])-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    display(Image(data=bgr8_to_jpeg(image)))
    time.sleep(0.05)


clear_output(wait = True)
image = np.array(camera.value)
gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

The first few lines are similar to what we accomplished in Module 3. We take the image captured by the camera and convert it to a grayscale image.

detections = detector.detect(gray_frame)

This line is the main purpose of the april tags library we imported. The "detector", which we created above when we first imported the package, is an object/class. The "detect" is a method within this class that takes an image and returns a list of april tags detected in the image. Each april tag is stored in its own custom format, with features that define both what it is and where it is (this will be shown later).

for detection in detections:

Since the return is a list of april tags detected, we can go through each detected april tag in this for loop.

corners = detection.corners
corners = [(int(x), int(y)) for (x, y) in corners]

One of the specific ways an apriltag is stored is with locating where the four corners are within the image frame. These two lines save those corner values, and then convert them into integers (just for ease of imaging). It is important to note that they are stored as purely x and y coordinates within the frame of the image, and not in the actual world. Therefore, these values refer to the specific pixels of the image for their location.

cv2.polylines(image, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)

This line uses cv2 to create a box that can be seen visually when we look at the image within our jupyter notebook.

tag_id = detection.tag_id
cv2.putText(image, f"ID: {tag_id}", (int(corners[0][0]), int(corners[0][1])-10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

This is one of the other pieces of data stored within the april tag. Because there are different patterns that an AprilTag can encompass, they each have their own unique ID to denote which pattern is which. If you try different AprilTags, you will notice they all have their own unique IDs that show on the image. The function cv2.putText is used to display these values on the image so we can see visually what tag it is finding.

display(Image(data=bgr8_to_jpeg(image)))
time.sleep(0.05)

Same as in Module 3, this converts the image to jpeg and displays, while the time.sleep sets the framerate.